In [1]:
%cd ../

/u02/thanh/workplace/plasma


In [2]:
import plasma.functional as F

from plasma.meta.object_graph import ContextManager, Inputs

In [3]:
class Handler(F.AutoPipe):

    def __init__(self, name, age, test=5):
        super().__init__()
        self.name = name
        self.age = age
        self.test = test

In [4]:
class Flow(F.AutoPipe):

    def __init__(self, handler, bee) -> None:
        super().__init__()
        self.handler = handler

In [5]:
class Service(F.AutoPipe):

    def __init__(self, flow:Flow, handler:Handler, test_factory:dict, check:int) -> None:
        super().__init__()
        print(test_factory)
        self.flow = flow
        self.handler = handler

In [6]:
class AnotherHandler(F.AutoPipe):
    
    def __init__(self, test_factory:dict, check:int, check2:int):
        super().__init__()
        
        self.test_factory = test_factory

In [7]:
manager =  ContextManager()
manager

In [8]:
service_manager = manager\
    .context('service_components')\
        .add_dependency('main', Service)\
        .add_dependency('another_handler', AnotherHandler)

service_manager

service_components
  |-> main
    |-> flow: Flow
    |-> handler: Handler
    |-> test_factory: dict
    |-> check: int
------------------------------------------
service_components
  |-> another_handler
    |-> test_factory
    |-> check
    |-> check2: int
------------------------------------------

In [9]:
factory = service_manager.factory('test_factory')
factory[8, 6] = 'hehe'

service_manager

service_components
  |-> main
    |-> flow: Flow
    |-> handler: Handler
    |-> test_factory: DependencyFactory
      |-> (8, 6) = str
    |-> check: int
------------------------------------------
service_components
  |-> another_handler
    |-> test_factory...
    |-> check
    |-> check2: int
------------------------------------------

In [10]:
service_manager.inputs('main', 'another_handler')

{
    check: Required,
    check2: Required,
    flow: Required,
    handler: Required,
}

In [11]:
package_manager = manager\
    .context('package')\
        .add_dependency('flow', Flow)\
        .add_dependency('handler', Handler)
package_manager           

package
  |-> flow
    |-> handler
      |-> name
      |-> age
      |-> test = int
    |-> bee
------------------------------------------

In [12]:
service_manager.link_name(package_manager)

service_components
  |-> main
    |-> flow: Flow --> package.flow...
    |-> handler: Handler --> package.handler...
    |-> test_factory: DependencyFactory
      |-> (8, 6) = str
    |-> check: int
------------------------------------------
service_components
  |-> another_handler
    |-> test_factory...
    |-> check
    |-> check2: int
------------------------------------------

In [13]:
requirements = service_manager.inputs('main', 'another_handler')
requirements

{
    package: {
        age: Required,
        bee: Required,
        name: Required,
    },
    check: Required,
    check2: Required,
}

In [14]:
class TestInputs2:...


class TestInputs(Inputs):
    check:int
    check2:int
    package:TestInputs2


class TestInputs2(Inputs):
    age:int
    bee:str
    name:str

In [15]:
inputs = TestInputs({'check': 8, 'check2': 12, 'package': {'age': 32, 'bee': 'buzzbuzz', 'name': 'tèo'}})
inputs 

{
  check: 8,
  check2: 12,
  package: {
    age: 32,
    bee: "buzzbuzz",
    name: "tèo"
  }
}

In [16]:
main, handler = inputs.init(service_manager, 'main', 'another_handler')

service_components
  |-> main
    |-> flow: Flow --> package.flow...
    |-> handler: Handler --> package.handler...
    |-> test_factory: DependencyFactory
      |-> (8, 6) = str
    |-> check: int
------------------------------------------
service_components
  |-> another_handler
    |-> test_factory...
    |-> check
    |-> check2: int
------------------------------------------
{(8, 6): 'hehe'}


In [17]:
main

Service(
  flow=Flow(
    handler=Handler(
      name='tèo',
      age=32,
      test=5,
    ),
  ),
  handler=Handler(
    name='tèo',
    age=32,
    test=5,
  ),
)

In [18]:
handler

AnotherHandler(
  test_factory={(8, 6): 'hehe'},
)